# G only KEGG based network architecture

> 

In [1]:
import numpy as np
import pandas as pd

# from EnvDL.core import ensure_dir_path_exists 
from EnvDL.dlfn import g2fc_datawrapper#, BigDataset, plDNN_general
# from EnvDL.dlfn import ResNet2d, BasicBlock2d
# from EnvDL.dlfn import LSUV_

import torch
import torch.nn.functional as F # F.mse_loss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger

# from EnvDL.dlfn import kegg_connections_build, kegg_connections_clean, kegg_connections_append_y_hat, kegg_connections_sanitize_names
# from EnvDL.dlfn import VNNHelper, VisableNeuralNetwork, Linear_block_reps
# from EnvDL.dlfn import plDNN_general, BigDataset
# from EnvDL.dlfn import reverse_edge_dict, reverse_node_props
# from EnvDL.dlfn import VNNVAEHelper, plVNNVAE
# from EnvDL.dlfn import kegg_connections_build, kegg_connections_clean, kegg_connections_append_y_hat, kegg_connections_sanitize_names
# from EnvDL.dlfn import VNNHelper, VisableNeuralNetwork, Linear_block_reps
# from EnvDL.dlfn import ListDataset, plVNN
# from EnvDL.dlfn import plDNN_general, BigDataset

In [2]:
import plotly.express as px

In [3]:
from sparsevnn.core import *
from sparsevnn.qol  import ensure_dir_path_exists
from sparsevnn.kegg import kegg_connections_build, kegg_connections_clean, kegg_connections_append_y_hat, kegg_connections_sanitize_names

In [4]:
cache_path = '../nbs_artifacts/02.40_g2fc_G_ACGT_sparsevnn_baseline/'
save_prefix = [e for e in cache_path.split('/') if e != ''][-1]

# Run settings: 
max_epoch  = 200
batch_size = 48
batch_size = 256

# VNN settings:
default_out_nodes_inp   = 4 #  4
default_out_nodes_edge  = 2 # 32
default_out_nodes_out   = 1

default_drop_nodes_inp  = 0.0
default_drop_nodes_edge = 0.
default_drop_nodes_out  = 0.0

default_reps_nodes_inp  = 1
default_reps_nodes_edge = 1
default_reps_nodes_out  = 1

max_epoch  = 20

In [5]:
use_gpu_num = 0

device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


In [6]:
ensure_dir_path_exists(dir_path = cache_path)

In [7]:
# from EnvDL.dlfn import kegg_connections_build, kegg_connections_clean, kegg_connections_append_y_hat, kegg_connections_sanitize_names
# from EnvDL.dlfn import VNNHelper, VisableNeuralNetwork, Linear_block_reps
# from EnvDL.dlfn import ListDataset, plVNN

## Fit Using VNNHelper

In [8]:

# Same setup as above to create kegg_gene_brite
X = g2fc_datawrapper()
X.set_split()
X.load_all(name_list = ['obs_geno_lookup', 'YMat', 'KEGG_slices',], store=True) 
X.calc_cs('YMat', version = 'np', filter = 'val:train')
ACGT_gene_slice_list =     X.get('KEGG_slices', ops_string='')
parsed_kegg_gene_entries = X.get('KEGG_entries')


# Restrict to only those with pathway
kegg_gene_brite = [e for e in parsed_kegg_gene_entries if 'BRITE' in e.keys()]

# also require to have a non-empty path
kegg_gene_brite = [e for e in kegg_gene_brite if not e['BRITE']['BRITE_PATHS'] == []]

print('Retaining '+ str(round(len(kegg_gene_brite)/len(parsed_kegg_gene_entries), 4)*100)+'%, '+str(len(kegg_gene_brite)
    )+'/'+str(len(parsed_kegg_gene_entries)
    )+' Entries'
    )
# kegg_gene_brite[1]['BRITE']['BRITE_PATHS']

Loading and storing default `phno`.


Retaining 43.53%, 6067/13939 Entries


In [9]:
kegg_connections = kegg_connections_build(kegg_gene_brite = kegg_gene_brite, 
                                          n_genes = 6067) 
kegg_connections = kegg_connections_clean(         kegg_connections = kegg_connections)
kegg_connections = kegg_connections_append_y_hat(  kegg_connections = kegg_connections)
kegg_connections = kegg_connections_sanitize_names(kegg_connections = kegg_connections, 
                                                   replace_chars = {'.':'_'})

Removed node "Others"


In [10]:
# initialize helper for input nodes
myvnn = VNNHelper(edge_dict = kegg_connections)

myvnn.nodes_inp[0:10]

# Get a mapping of brite names to tensor list index
find_names = myvnn.nodes_inp # e.g. ['100383860', '100278565', ... ]
lookup_dict = {}

# the only difference lookup_dict and brite_node_to_list_idx_dict above is that this is made using the full set of genes in the list 
# whereas that is made using kegg_gene_brite which is a subset
for i in range(len(parsed_kegg_gene_entries)):
    if 'BRITE' not in parsed_kegg_gene_entries[i].keys():
        pass
    elif parsed_kegg_gene_entries[i]['BRITE']['BRITE_PATHS'] == []:
        pass
    else:
        name = parsed_kegg_gene_entries[i]['BRITE']['BRITE_PATHS'][0][-1]
        if name in find_names:
            lookup_dict[name] = i
# lookup_dict    

In [11]:
# # if permuting gene identities
# torch.manual_seed(5461)

# keys = [e for e in lookup_dict.keys()]

# # vals = [lookup_dict[e] for e in lookup_dict.keys()]
# # dict(zip(keys, [int(i) for i in torch.randperm(len(keys))]))

# idx = torch.tensor([lookup_dict[e] for e in myvnn.nodes_inp])
# idx = idx[torch.randperm(idx.shape[0])]
# idx = [int(i) for i in idx]
# temp = dict(zip(myvnn.nodes_inp, idx))

# randomized_lookup_dict = {}
# for e in lookup_dict.keys():
#     if e not in temp.keys():
#         randomized_lookup_dict[e] = lookup_dict[e]
#     else:
#         randomized_lookup_dict[e] = temp[e]

# lookup_dict = randomized_lookup_dict

In [12]:
brite_node_to_list_idx_dict = {}
for i in range(len(kegg_gene_brite)):
    brite_node_to_list_idx_dict[str(kegg_gene_brite[i]['BRITE']['BRITE_PATHS'][0][-1])] = i        

# Get the input sizes for the graph
size_in_zip = zip(myvnn.nodes_inp, [np.prod(ACGT_gene_slice_list[lookup_dict[e]].shape[1:]) for e  in myvnn.nodes_inp])


In [13]:

# init input node sizes
myvnn.set_node_props(key = 'inp', node_val_zip = size_in_zip)

# init node output sizes
myvnn.set_node_props(key = 'out', node_val_zip = zip(myvnn.nodes_inp, [default_out_nodes_inp  for e in myvnn.nodes_inp]))
myvnn.set_node_props(key = 'out', node_val_zip = zip(myvnn.nodes_edge,[default_out_nodes_edge for e in myvnn.nodes_edge]))
myvnn.set_node_props(key = 'out', node_val_zip = zip(myvnn.nodes_out, [default_out_nodes_out  for e in myvnn.nodes_out]))


# # options should be controlled by node_props
myvnn.set_node_props(key = 'flatten', node_val_zip = zip(
    myvnn.nodes_inp, 
    [True for e in myvnn.nodes_inp]))

myvnn.set_node_props(key = 'reps', node_val_zip = zip(myvnn.nodes_inp, [default_reps_nodes_inp  for e in myvnn.nodes_inp]))
myvnn.set_node_props(key = 'reps', node_val_zip = zip(myvnn.nodes_edge,[default_reps_nodes_edge for e in myvnn.nodes_edge]))
myvnn.set_node_props(key = 'reps', node_val_zip = zip(myvnn.nodes_out, [default_reps_nodes_out  for e in myvnn.nodes_out]))

myvnn.set_node_props(key = 'drop', node_val_zip = zip(myvnn.nodes_inp, [default_drop_nodes_inp  for e in myvnn.nodes_inp]))
myvnn.set_node_props(key = 'drop', node_val_zip = zip(myvnn.nodes_edge,[default_drop_nodes_edge for e in myvnn.nodes_edge]))
myvnn.set_node_props(key = 'drop', node_val_zip = zip(myvnn.nodes_out, [default_drop_nodes_out  for e in myvnn.nodes_out]))

# init edge node input size (propagate forward input/edge outpus)
myvnn.calc_edge_inp()

# myvnn.mk_digraph(include = ['node_name', 'inp_size', 'out_size'])
# myvnn.mk_digraph(include = [''])

In [14]:
from EnvDL.dlfn import plDNN_general, BigDataset

In [15]:
vals = X.get('KEGG_slices', ops_string='asarray from_numpy float')

In [16]:
# restrict to the tensors that will be used
vals = [vals[lookup_dict[i]] for i in myvnn.nodes_inp] #TODO CONFIRM.
# send to gpu
# vals = [val.to('cuda') for val in vals]

In [17]:
# replace lookup so that it matches the lenght of the input tensors
new_lookup_dict = {}
for i in range(len(myvnn.nodes_inp)):
    new_lookup_dict[myvnn.nodes_inp[i]] = i
    # print((myvnn.nodes_inp[i], i))
    # break

In [18]:
## start insert

### Calculate nodes membership in each matrix and positions within each

In [19]:
node_props = myvnn.node_props
# Linear_block = Linear_block_reps,
edge_dict = myvnn.edge_dict
dependancy_order = myvnn.dependancy_order
node_to_inp_num_dict = new_lookup_dict

In [20]:
# check dep order

tally = []
for d in dependancy_order:
    if edge_dict[d] == []:
        tally.append(d)
    elif False not in [True if e in tally else False for e in edge_dict[d]]:
        tally.append(d)
    else:
        print('error!')
        break

In [21]:
# build output nodes 
d_out = {0:[]}
for d in dependancy_order:
    if edge_dict[d] == []:
        d_out[min(d_out.keys())].append(d)
    else:
        # print((d, edge_dict[d]))

        d_out_i = 1+max(sum([[key for key in d_out.keys() if e in d_out[key]]
                   for e in edge_dict[d]], []))
        
        if d_out_i not in d_out.keys():
            d_out[d_out_i] = []
        d_out[d_out_i].append(d)

In [22]:
# build input nodes NOPE. THE PASSHTROUGHS! 
d_eye = {}
tally = []
for i in range(max(d_out.keys()), min(d_out.keys()), -1):
    # print(i)
    nodes_needed = sum([edge_dict[e] for e in d_out[i]], [])+tally
    # check against what is there and then dedupe
    nodes_needed = [e for e in nodes_needed if e not in d_out[i-1]]
    nodes_needed = list(set(nodes_needed))
    tally = nodes_needed
    d_eye[i] = nodes_needed

# d_inp[0]= d_out[0]
    
[len(d_eye[i]) for i in d_eye.keys()]

[58, 55, 106, 89, 105, 43, 0]

In [23]:
[(key, len(d_out[key])) for key in d_out.keys()]

[(0, 6067), (1, 2167), (2, 447), (3, 143), (4, 25), (5, 8), (6, 10), (7, 1)]

In [24]:
dd = {}
for i in d_eye.keys():
    dd[i] = {'out': d_out[i],
             'inp': d_out[i-1],
             'eye': d_eye[i]}
# plus special 0 layer that handles the snps
    
dd[0] = {'out': d_out[0],
         'inp': d_out[0],
         'eye': []}

In [25]:
# check that the output nodes' inputs are satisfied by the same layer's inputs (inp and eye)

for i in dd.keys():
    # out node in each
    for e in dd[i]['out']:
        # node depends in inp/eye
        node_pass_list = [True if ee in dd[i]['inp']+dd[i]['eye'] else False 
                          for ee in edge_dict[e]]
        if False not in node_pass_list:
            pass
        else:
            print('exit') 

In [26]:
print("Layer\t#In\t#Out")
for i in range(min(dd.keys()), max(dd.keys())+1, 1):
    node_in      = [node_props[e]['out'] for e in dd[i]['inp']+dd[i  ]['eye'] ]
    if i == max(dd.keys()):
        node_out = [node_props[e]['out'] for e in dd[i]['out'] ]
    else:
        node_out = [node_props[e]['out'] for e in dd[i]['out']+dd[i+1]['eye']]
    print(f'{i}:\t{sum(node_in)}\t{sum(node_out)}')

Layer	#In	#Out
0:	24268	24268
1:	24268	4506
2:	4506	1104
3:	1104	464
4:	464	262
5:	262	126
6:	126	136
7:	136	1


### Creating Structured Matrices for Layers

In [27]:
dd.keys()

dict_keys([7, 6, 5, 4, 3, 2, 1, 0])

In [28]:
M_list = [structured_layer_info(i = ii, node_groups = dd, node_props= node_props, edge_dict = edge_dict, as_sparse=True) for ii in range(0, max(dd.keys())+1)]

In [29]:
[e.weight.shape for e in M_list]

[torch.Size([24268, 144468]),
 torch.Size([4506, 24268]),
 torch.Size([1104, 4506]),
 torch.Size([464, 1104]),
 torch.Size([262, 464]),
 torch.Size([126, 262]),
 torch.Size([136, 126]),
 torch.Size([1, 136])]

### Setup Dataloader using `M_list`

In [30]:
vals = X.get('KEGG_slices', ops_string='asarray from_numpy float')
# restrict to the tensors that will be used
vals = torch.concat([vals[lookup_dict[i]].reshape(4926, -1) 
                     for i in M_list[0].row_inp
                    #  for i in dd[0]['inp'] # matches
                     ], axis = 1)
vals.shape
vals = vals.to('cuda')

In [31]:
training_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = True,
    lookup_obs =  X.get('val:train',       ops_string='                   asarray from_numpy'), 
    lookup_geno = X.get('obs_geno_lookup', ops_string='   filter:val:train asarray from_numpy'),
    y =           X.get('YMat',            ops_string='cs filter:val:train asarray from_numpy float cuda:0')[:, None],
    # y =           X.get('YMat',            ops_string='cs filter:val:train asarray from_numpy float')[:, None],
    G =           vals,
    G_type = 'raw',
    # send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = True
)

validation_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = True,
    lookup_obs =  X.get('val:test',        ops_string='                   asarray from_numpy'), 
    lookup_geno = X.get('obs_geno_lookup', ops_string='   filter:val:test asarray from_numpy'),
    y =           X.get('YMat',            ops_string='cs filter:val:test asarray from_numpy float cuda:0')[:, None],
    G =           vals,
    G_type = 'raw',
    # send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = False
)


In [32]:
# Version to predict enviromental residuals?

In [33]:
# load_from = '../nbs_artifacts/01.03_g2fc_prep_matrices/'
# load_from = '../nbs_artifacts/01.03_g2fc_prep_matrices/'
# phno_geno = pd.read_csv(load_from+'phno_geno.csv')
# phno = phno_geno


# obs_geno_lookup = np.load(load_from+'obs_geno_lookup.npy') # Phno_Idx  Geno_Idx  Is_Phno_Idx
# obs_env_lookup = np.load(load_from+'obs_env_lookup.npy')   # Phno_Idx  Env_Idx   Is_Phno_Idx
# YMat = np.load(load_from+'YMat.npy')

In [34]:
# # from EnvDL.dlfn import * 
# from EnvDL.dlfn import read_split_info, find_idxs_split_dict, calc_cs, apply_cs

In [35]:

# ## Create train/test validate indicies from json
# load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/'

# split_info = read_split_info(
#     load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/',
#     json_prefix = '2023:9:5:12:8:26')

# temp = phno.copy()
# temp[['Female', 'Male']] = temp['Hybrid'].str.split('/', expand = True)

# test_dict = find_idxs_split_dict(
#     obs_df = temp, 
#     split_dict = split_info['test'][0]
# )
# # test_dict

# # since this is applying predefined model structure no need for validation.
# # This is included for my future reference when validation is needed.
# temp = temp.loc[test_dict['train_idx'], ] # restrict before re-aplying

# val_dict = find_idxs_split_dict(
#     obs_df = temp, 
#     split_dict = split_info['validate'][0]
# )
# # val_dict

# # test_dict

# train_idx = test_dict['train_idx']
# test_idx  = test_dict['test_idx']

In [36]:
from tqdm import tqdm

In [37]:
# # Process data to get env means
# # obs_env_lookup   # Phno_Idx  Env_Idx   Is_Phno_Idx

# YMat_EnvMean = YMat.copy()

# for i in tqdm(list(set(obs_env_lookup[:, 1]))):
#     mask = (obs_env_lookup[:, 1] == i)
#     YMat_EnvMean[mask] = YMat_EnvMean[mask].mean()

In [38]:
# # subtract to get residuals
# YMat = YMat - YMat_EnvMean
# # proceed as normal...

In [39]:
# YMat_cs = calc_cs(YMat[train_idx])
# y_cs = apply_cs(YMat, YMat_cs)

In [40]:
# y_temp = torch.from_numpy(y_cs).to(torch.float)#[:, None]

In [41]:
# training_dataloader = DataLoader(BigDataset(
#     lookups_are_filtered = True,
#     lookup_obs =  X.get('val:train',       ops_string='                   asarray from_numpy'), 
#     lookup_geno = X.get('obs_geno_lookup', ops_string='   filter:val:train asarray from_numpy'),
#     y =           y_temp[train_idx][:, None].to('cuda'),
#     G =           vals,
#     G_type = 'raw',
#     # send_batch_to_gpu = 'cuda:0'
#     ),
#     batch_size = batch_size,
#     shuffle = True
# )

# validation_dataloader = DataLoader(BigDataset(
#     lookups_are_filtered = True,
#     lookup_obs =  X.get('val:test',        ops_string='                   asarray from_numpy'), 
#     lookup_geno = X.get('obs_geno_lookup', ops_string='   filter:val:test asarray from_numpy'),
#     y =           y_temp[test_idx][:, None].to('cuda'),
#     G =           vals,
#     G_type = 'raw',
#     # send_batch_to_gpu = 'cuda:0'
#     ),
#     batch_size = batch_size,
#     shuffle = False
# )


In [42]:
# training_dataloader = DataLoader(BigDataset(
#     lookups_are_filtered = True,
#     lookup_obs =  X.get('val:train',       ops_string='                   asarray from_numpy'), 
#     lookup_geno = X.get('obs_geno_lookup', ops_string='   filter:test:train asarray from_numpy'),
#     y =           X.get('YMat',            ops_string='cs filter:test:train asarray from_numpy float cuda:0')[:, None],
#     # y =           X.get('YMat',            ops_string='cs filter:val:train asarray from_numpy float')[:, None],
#     G =           vals,
#     G_type = 'raw',
#     # send_batch_to_gpu = 'cuda:0'
#     ),
#     batch_size = batch_size,
#     shuffle = True
# )

# validation_dataloader = DataLoader(BigDataset(
#     lookups_are_filtered = True,
#     lookup_obs =  X.get('val:test',        ops_string='                   asarray from_numpy'), 
#     lookup_geno = X.get('obs_geno_lookup', ops_string='   filter:test:test asarray from_numpy'),
#     y =           X.get('YMat',            ops_string='cs filter:test:test asarray from_numpy float cuda:0')[:, None],
#     G =           vals,
#     G_type = 'raw',
#     # send_batch_to_gpu = 'cuda:0'
#     ),
#     batch_size = batch_size,
#     shuffle = False
# )


## Structured Layer

In [43]:
class NeuralNetwork(nn.Module):
    def __init__(self, layer_list):
        super(NeuralNetwork, self).__init__()
        self.layer_list = nn.ModuleList(layer_list)
 
    def forward(self, x):
        for l in self.layer_list:
            x = l(x)
        return x

layer_list = []
for i in range(len(M_list)):
    
    apply_relu = None
    if i+1 != len(M_list): # apply relu to all but the last layer
        apply_relu = F.relu
    

    l = SparseLinearCustom(
        M_list[i].weight.shape[1], # have to transpose this?
        M_list[i].weight.shape[0],
        connectivity   = torch.LongTensor(M_list[i].weight.coalesce().indices()),
        custom_weights = M_list[i].weight.coalesce().values(), 
        custom_bias    = M_list[i].bias.clone().detach(), 
        weight_grad_bool = M_list[i].weight_grad_bool, 
        bias_grad_bool   = M_list[i].bias_grad_bool, #.to_sparse()#.indices()
        dropout_p        = M_list[i].dropout_p,
        nonlinear_transform= apply_relu
        )

    layer_list += [l]

model = NeuralNetwork(layer_list)

In [44]:
# Standard Training

# VNN = plDNN_general(model)  

# optimizer = VNN.configure_optimizers()

# logger = TensorBoardLogger("tb_vnnsparse_logs", name='tb_sparsevnn_baseline')
# trainer = pl.Trainer(max_epoch=max_epoch, logger=logger)

# trainer.fit(model=VNN, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)


# Intepretability

In [45]:
# set up gradients for testing

yi, xi = next(iter(training_dataloader))
yi = yi.to('cpu')
xi = xi.to('cpu')


optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.MSELoss()

loss = loss_fn(model(xi), yi)
optimizer.zero_grad()
loss.backward()

In [46]:
# find indices in a structured matrix that correspond to the in/out of a given node. 

def find_node_in_layer_info(
    layer_info_list:list, # List where all elements are `sparsevnn.core.structured_layer_info`. 
    edge_dict:dict, # dictionary of graph edges. Needed to find the correct input dim. indices of the `sparsevnn.core.SparseLinearCustom`. 
    query:str, # name of the node to be identified
):
    # find the right index in the list:
    list_idx = [i for i in range(len(layer_info_list)) if query in layer_info_list[i].col_out]
    assert len(list_idx) == 1
    list_idx = list_idx[0]

    # if an edge or output
    if edge_dict[query] != []:
        rows_info = {e : layer_info_list[list_idx].row_info[e] for  e in edge_dict[query]}
    else: # if input
        rows_info = {e : layer_info_list[list_idx].row_info[e] for  e in [query]}

    return {
        'list_idx':  list_idx,
        'col_info':  layer_info_list[list_idx].col_info[query],
        'rows_info': rows_info
        }


In [47]:
find_node_in_layer_info(
    layer_info_list = M_list,
    edge_dict=edge_dict,
    query = 'ProteinPhosphatasesAndAssociatedProteins[Br-Zma01009]'        
)

{'list_idx': 5,
 'col_info': {'size': tensor(2, dtype=torch.int32),
  'stop': tensor(10),
  'start': tensor(8)},
 'rows_info': {'ProteinSerine/ThreoninePhosphatases': {'size': tensor(2, dtype=torch.int32),
   'stop': tensor(20),
   'start': tensor(18)},
  'ProteinTyrosinePhosphatases(Ptps)': {'size': tensor(2, dtype=torch.int32),
   'stop': tensor(124),
   'start': tensor(122)},
  'HadPhosphatases': {'size': tensor(2, dtype=torch.int32),
   'stop': tensor(216),
   'start': tensor(214)}}}

In [48]:
import sparsevnn

In [49]:
def slice_SparseLinearCustom(
        slc:sparsevnn.core.SparseLinearCustom, # SparseLinearCustom layer
        inp_min:int = None, # Start slice index for input. See the `structured_layer_info`'s attribute `.row_info['start']`
        inp_max:int = None, # Stop slice index for input. See the `structured_layer_info`'s attribute `.row_info['stop']`
        out_min:int = None, # Start slice index for output. See the `structured_layer_info`'s attribute `.col_info['start']`
        out_max:int = None, # Stop slice index for output. See the `structured_layer_info`'s attribute `.col_info['stop']`
        weight_or_bias:str = 'weight', # Specifies if weights or bias should be considered.
        grad:bool = False, # Return parameters or gradients from `slc`
        ):    
    "Return subset of a `SparseLinearCustom` layers weights or bias. Retrieve either the parameter values or the gradient."
    connectivity = slc.connectivity

    if weight_or_bias == 'bias':
            if grad == False:
                    return slc.bias.grad[out_min:out_max]
            if grad: 
                    return slc.bias[out_min:out_max]

    if weight_or_bias == 'weight':
        # 1 is in features
        inp_select_bool = (slc.connectivity[1] >= inp_min) & (slc.connectivity[1] <= inp_max)
        out_select_bool = (slc.connectivity[0] >= out_min) & (slc.connectivity[0] <= out_max)

        select_bool = inp_select_bool & out_select_bool

        new_con0 = slc.connectivity[0][select_bool]
        new_con1 = slc.connectivity[1][select_bool]
        new_con0 -= new_con0.min()
        new_con1 -= new_con1.min()

        i = torch.concat([
        new_con0[None, :],
        new_con1[None, :]], axis = 0)


        if grad == False:
              v = slc.weights[select_bool]

        if grad == True:
              v = slc.weights.grad[select_bool]

        out_shape = (
              int(1+new_con0.max()),
              int(1+new_con1.max())
              )


        return torch.sparse_coo_tensor(indices = i, 
                                       values = v, 
                                       size = out_shape
                                       ).to_dense()





In [50]:

# input
node_info = find_node_in_layer_info(
    layer_info_list = M_list,
    edge_dict=edge_dict,
    query = '103646242'
)


# edge 
# node_info = find_node_in_layer_info(
#     layer_info_list = M_list,
#     edge_dict=edge_dict,
#     query = 'ProteinPhosphatasesAndAssociatedProteins[Br-Zma01009]'        
# )

node_info = find_node_in_layer_info(
    layer_info_list = M_list,
    edge_dict=edge_dict,
    query = 'y_hat'        
)

node_info

{'list_idx': 7,
 'col_info': {'size': tensor(1, dtype=torch.int32),
  'stop': tensor(1),
  'start': tensor(0)},
 'rows_info': {'KeggOrthology(Ko)[Br-Zma00001]': {'size': tensor(2, dtype=torch.int32),
   'stop': tensor(34),
   'start': tensor(32)},
  'Enzymes[Br-Zma01000]': {'size': tensor(2, dtype=torch.int32),
   'stop': tensor(128),
   'start': tensor(126)},
  'TransferRnaBiogenesis[Br-Zma03016]': {'size': tensor(2, dtype=torch.int32),
   'stop': tensor(20),
   'start': tensor(18)},
  'ChromosomeAndAssociatedProteins[Br-Zma03036]': {'size': tensor(2, dtype=torch.int32),
   'stop': tensor(18),
   'start': tensor(16)},
  'MessengerRnaBiogenesis[Br-Zma03019]': {'size': tensor(2, dtype=torch.int32),
   'stop': tensor(16),
   'start': tensor(14)},
  'SecretionSystem[Br-Zma02044]': {'size': tensor(2, dtype=torch.int32),
   'stop': tensor(94),
   'start': tensor(92)},
  'Transporters[Br-Zma02000]': {'size': tensor(2, dtype=torch.int32),
   'stop': tensor(36),
   'start': tensor(34)},
  'Ubi

In [51]:
def get_node_slice_SparseLinearCustom(
    # for `find_node_in_layer_info`
    layer_info_list:list, # List where all elements are `sparsevnn.core.structured_layer_info`. 
    edge_dict:dict, # dictionary of graph edges. Needed to find the correct input dim. indices of the `sparsevnn.core.SparseLinearCustom`. 
    query:str, # name of the node to be identified
    # additional parameter for `slice_SparseLinearCustom`
    neural_network_layer_list, # Neural networks's ModuleList which contains `SparseLinearCustom` layers
    weight_or_bias:str = 'weight', # Specifies if weights or bias should be considered.
    grad:bool = False, # Return parameters or gradients from `slc`
    ):
    "Wraps `find_node_in_layer_info` and `slice_SparseLinearCustom` to get all the requested values for each edge"
    node_info = find_node_in_layer_info(
        layer_info_list = layer_info_list,
        edge_dict=edge_dict,
        query = query
        )
    
    out_list = []
    out_list_nodes = []
    in_nodes = node_info['rows_info'].keys()
    for in_node in in_nodes:
        e = node_info['rows_info'][in_node]
        e_out = slice_SparseLinearCustom(
                slc = neural_network_layer_list[node_info['list_idx']],
                inp_min = e['start'],
                inp_max = e['stop'],
                out_min = node_info['col_info']['start'],
                out_max = node_info['col_info']['stop'],
                weight_or_bias = weight_or_bias, 
                grad = grad
                )
        
        out_list += [e_out]
        out_list_nodes += [in_node]

    return {
        'node_names': out_list_nodes, 
        'node_vals': out_list
    }


In [52]:
get_node_slice_SparseLinearCustom(
    # for `find_node_in_layer_info`
    layer_info_list = M_list,
    edge_dict = edge_dict,
    query = 'y_hat',
    # additional parameter for `slice_SparseLinearCustom`
    neural_network_layer_list = model.layer_list,
    weight_or_bias = 'weight', 
    grad = False
    )

{'node_names': ['KeggOrthology(Ko)[Br-Zma00001]',
  'Enzymes[Br-Zma01000]',
  'TransferRnaBiogenesis[Br-Zma03016]',
  'ChromosomeAndAssociatedProteins[Br-Zma03036]',
  'MessengerRnaBiogenesis[Br-Zma03019]',
  'SecretionSystem[Br-Zma02044]',
  'Transporters[Br-Zma02000]',
  'UbiquitinSystem[Br-Zma04121]',
  'MembraneTrafficking[Br-Zma04131]',
  'Exosome[Br-Zma04147]',
  'TranscriptionFactors[Br-Zma03000]',
  'Spliceosome[Br-Zma03041]',
  'CytoskeletonProteins[Br-Zma04812]',
  'MitochondrialBiogenesis[Br-Zma03029]',
  'ChaperonesAndFoldingCatalysts[Br-Zma03110]',
  'CiliumAndAssociatedProteins[Br-Zma03037]',
  'TranscriptionMachinery[Br-Zma03021]',
  'Glycosyltransferases[Br-Zma01003]',
  'PhotosynthesisProteins[Br-Zma00194]',
  'Ribosome[Br-Zma03011]',
  'DnaReplicationProteins[Br-Zma03032]',
  'DnaRepairAndRecombinationProteins[Br-Zma03400]',
  'ProteinKinases[Br-Zma01001]',
  'PlantMloReceptors',
  'ProteinPhosphatasesAndAssociatedProteins[Br-Zma01009]',
  'Proteasome[Br-Zma03051]',
 

In [53]:
stophere

NameError: name 'stophere' is not defined

In [ ]:
# y_i, xs_i = next(iter(training_dataloader))

In [ ]:
# loss_fn = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# from tqdm import tqdm

In [ ]:
# for ii in tqdm(range(2)):
#     pred = model(xs_i)
#     loss = loss_fn(pred, y_i)
#     if ii % 100 == 0:
#         print(f'{loss}')
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

In [ ]:
# # After training are unity weights still unity?
# i = 2
# (
# M_list[i].bias_eye_bool, 
# M_list[i].weight_eye_bool,
# model.layer_list[((2*(i)))].weight,
# model.layer_list[((2*(i)))].weight.grad
# )

In [ ]:
# dd

In [ ]:
# M.row_inp
# M.row_eye
# M.col_out
# M.col_eye

# M.row_info
# # M.col_info

# # M.weight
# # M.weight_bool
# # M.weight_eye_bool
# # M.bias
# # M.bias_eye_bool

In [ ]:
# query = list(M_list[i].col_info.keys())[0]
# slice_accumulator = []
# # M_list[i].row_info[query]
# c1 = M_list[i].col_info[query]['start'] 
# c2 = M_list[i].col_info[query]['stop']

# # could get full slice then drop zero values too
# for e in edge_dict[query]:
#     r1 = M_list[i].row_info[e]['start'] 
#     r2 = M_list[i].row_info[e]['stop']
#     slice_accumulator += [model.layer_list[((2*(i)))].weight.swapaxes(0,1)[r1:r2, c1:c2].clone().detach().requires_grad_(False)]

In [ ]:
# torch.concat(slice_accumulator)

In [ ]:
# model.layer_list[((2*(i)))].weight

In [ ]:
# dot

In [ ]:
# node_to_weights = {}
# for i in range(0, len(M_list)):
#     for query in M_list[i].col_out:
#         slice_accumulator = []
#         # M_list[i].row_info[query]
#         c1 = M_list[i].col_info[query]['start'] 
#         c2 = M_list[i].col_info[query]['stop']

#         # could get full slice then drop zero values too
#         if i == 0:
#             r1 = M_list[i].row_info[query]['start'] 
#             r2 = M_list[i].row_info[query]['stop']
#             slice_accumulator += [model.layer_list[((2*(i)))].weight.swapaxes(0,1)[r1:r2, c1:c2].clone().detach().requires_grad_(False)]

#         else: 
#             for e in edge_dict[query]:
#                 r1 = M_list[i].row_info[e]['start'] 
#                 r2 = M_list[i].row_info[e]['stop']
#                 slice_accumulator += [model.layer_list[((2*(i)))].weight.swapaxes(0,1)[r1:r2, c1:c2].clone().detach().requires_grad_(False)]

#         slice_accumulator = torch.concat(slice_accumulator)
#         node_to_weights[query] = slice_accumulator

In [ ]:
# xx = [np.round(float(node_to_weights[key].abs().mean()), 3) for key in node_to_weights.keys()]
# color_vals = ['#ffffff', '#fff7ec', '#fee8c8', '#fdd49e', '#fdbb84', '#fc8d59', '#ef6548', '#d7301f', '#b30000'#, '#7f0000'
#               ]
# color_cutoffs = [i*max(xx)/len(color_vals) for i in range(len(color_vals))]


# dot = Digraph()
# for key in node_to_weights.keys():


#     key_mean_w = np.round(float(node_to_weights[key].abs().mean()), 3)
#     color_val = color_vals[[i for i in range(len(color_cutoffs)) if key_mean_w >= color_cutoffs[i]][-1]]

#     # key_label = name_cleanup(input = key, newline_char_threshold = 20)+'\nMean: '+str(key_mean_w)
#     # dot.node(key, key_label, style='filled', fillcolor=color_val)  
    
    
#     key_label = name_cleanup(input = key, newline_char_threshold = 20)+'\n           '  
#     dot.node(key, key_label)

#     if key in kegg_connections.keys():
#         for value in kegg_connections[key]:
#             # edge takes a head/tail whereas edges takes name pairs concatednated (A, B -> AB)in a list
#             dot.edge(value, key)    

# dot